In [17]:
import math as m
import numpy as np
import scipy.optimize

In [7]:
### PARAM: Initial player or pearl position
s_x0 = 0.5
s_y0 = 4
s_z0 = 0.5

s_y0 = s_y0 + 1.62 - 0.1

In [2]:
### Initial velocity at desired axis given a pitch. Yaw is kept constant to 360 degrees. 
def v_0(axis:str, pitch:float) -> float:
    return {
        "x": 0,
        "y": -1.5 * m.sin(pitch),
        "z": 1.5 * m.cos(pitch)
    }[axis]

In [6]:
### Component-wise velocity function given axis, initial pitch, and time. We keep yaw constant at 360 degrees
def v(axis:str, pitch:float, t:int) -> float:
    return {
        "x": v_0(axis, pitch)*0.99**m.floor(t),
        "y": v_0(axis, pitch)*0.99**m.floor(t) - 0.03*(100-100*0.99**m.floor(t)),
        "z": v_0(axis, pitch)*0.99**m.floor(t)
    }[axis]

In [8]:
### Component-wise position function. We keep yaw constant at 360 degrees (hence, zeroing every sin(yaw))
def s(axis:str, pitch:float, t:int) -> float:
    return {
        "x": s_x0,
        "y": s_y0 - 3*t - (1.5*m.sin(pitch) - 3) * (100-100**(1-t)*99**t),
        "z": s_z0 + 1.5*cos(pitch) * (100 - 100**(1-t)*99**t)
    }[axis]

In [16]:
### Test: Find the root of the y-position function
result = root_scalar(lambda x: x**2 - 4, x0=-10, x1=50)
result

      converged: True
           flag: 'converged'
 function_calls: 13
     iterations: 12
           root: -2.0